# Modelos de N-gramas

En un modelo de lenguaje la probabilidad condicional de una palabra $w_m$ dado un contexto $w_1w_2...w_{m-1}$, se puede calcular como:

$$P(w_m|w_1^{m-1}) = \frac{C(w_1w_2...w_m)}{∑_{w\in V} C(w_1w_2...w_{m-1}w)}$$

donde $V$ es el vocabulario del modelo.

Pero un modelo de N-gramas aproxima esa $P(w_m|w_1^{m-1})$ como:

$$P(w_m|w_1^{m-1}) ≈ P(w_m|w_{m-n+1}^{m-1}) $$

donde el N-grama es $(w_{m-n+1}w_{m-n+2}...w_{m-1}w_m)$

La idea intituiva de esta aproximación es que en vez de considerar todo el contexto anterior a una palabra, se considera solo las $N-1$ palabras anteriores.

Para calcular estas probabilidades se necesita de una colección de textos (corpus) de donde se recompilen los conteos de N-gramas y el vocabulario de palabras que usara el modelo, posteriormente se calcula una tabla de probabilidades por cada N-grama que aparezca en el corpus, una vez teniendo estas probabilidades, se pueden usar para estimar la probabilidad de una oración de la siguiente forma:

$$P(w_1^m) = P(w_1)P(w_2|w_1)P(w3|w_1^2)...P(w_m|w_1^{m-1})$$
$$ \approx 𝛱_{k=1}^m P(w_k|w_{k-n+1}^{k-1}) \qquad \qquad\qquad\,$$

Para que todas las probabilidades tengan contexto suficiente se agregan a la oración $N-1$ tokens especiales "\<s\>" que indican el inicio de oración, y un token especial "\<\s\>" que indica el final de la oración. Entonces para $N = 3$ la oración "*nunca fui popular*" se transforma en:

$$<s> \;\; <s> \;\; nunca \;\; fui \;\; popular \;\; <\backslash s>$$

#### Clases de Vocabulario y N-gramas

In [ ]:
from IPython.display import Markdown, display
import ipywidgets as widgets

import numpy as np
import pandas as pd
import random
from collections import Counter, OrderedDict
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def laplace(conteoGrande,conteoPeque,tamVocabulario):
    return (conteoGrande+1)/(conteoPeque+tamVocabulario)

class Vocabulario():
  def __init__(self,corpus):
    conteoVocabulario = {}
    for oracion in corpus:
      for i in range(0,len(oracion)):
        conteoVocabulario[oracion[i]] = conteoVocabulario.setdefault(oracion[i], 0) + 1

    self.palabras = pd.DataFrame.from_dict(conteoVocabulario,orient="index",columns=["Frecuencia"])
    self.palabras = self.palabras.sort_values("Frecuencia",ascending=False)
    self.palabras_unk = None

  def __len__(self):
    return len(self.palabras)

  def reducir_vocabulario(self,criterio,parametro):
    if criterio == "N palabras":
      self.palabras_unk = self.palabras.tail(len(self.palabras)-parametro).copy()
      self.palabras = self.palabras.head(parametro)
      self.palabras.loc["<UNK>"] = {"Frecuencia":self.palabras_unk["Frecuencia"].sum()}
      self.palabras.loc["<\s>"] = {"Frecuencia":0}
      self.palabras = self.palabras.sort_values("Frecuencia",ascending=False)

    else: # "FrecMin"
      self.palabras_unk = self.palabras[self.palabras.apply(lambda x: x["Frecuencia"] < parametro,axis=1)].copy()
      self.palabras = self.palabras[self.palabras.apply(lambda x: x["Frecuencia"] >= parametro,axis=1)].copy()
      self.palabras.loc["<UNK>"] = {"Frecuencia":self.palabras_unk["Frecuencia"].sum()}
      self.palabras.loc["<\s>"] = {"Frecuencia":0}
      self.palabras = self.palabras.sort_values("Frecuencia",ascending=False)

  def imprimir_datos_vocabulario(self):
    display(Markdown(f'**Número total de palabras diferentes en el vocabulario reducido:**'))
    print(len(self.palabras))
    display(Markdown(f'**10 palabras con mayor frecuencia:**'))
    print(self.palabras.iloc[:100])
    display(Markdown(f'**Mediana de las frecuencias de palabras**'))
    print(self.palabras.median()["Frecuencia"])
    display(Markdown(f'**10 palabras con menor frecuencia:**'))
    print(self.palabras.iloc[-11:-1])

    display(Markdown(f'**Grafica Rango y Frecuencia:**'))
    self.palabras['Rango'] = range(1,len(self.palabras)+1)
    plt.plot(self.palabras['Rango'],self.palabras['Frecuencia'],'bo',markersize=3)
    plt.yscale('log')
    plt.xscale('log')
    plt.xlabel("Rango")
    plt.ylabel("Frecuencia")
    plt.show()

class Ngrama():
  def __init__(self,n,vocabulario,suavizado = "Laplace"): # Suavizado = ["Laplace","Backoff"]
    self.n = n
    self.vocabulario = vocabulario
    self.Nmenos1Gramas = None #nMenos1Grama
    self.suavizado = suavizado

    self.numTokens = 0

    self.conteos = None
    self.probabilidades = None

  def transformar_oracion(self,oracion):
    # Transforma una oracion dependiendo del vocabulario administrado
    nueva_oracion = []
    for palabra in oracion:
      if palabra in self.vocabulario.palabras.index or palabra == "<s>" or palabra == "<\s>":
        nueva_oracion.append(palabra)
      else:
        nueva_oracion.append("<UNK>")
    return nueva_oracion

  def transformar_corpus(self,corpus):
    # Transforma todo un corpus dependiendo del vocabulario administrado
    corpus_transformado = []
    for oracion in corpus:
      oracion_transformada = self.transformar_oracion(oracion)
      corpus_transformado.append(oracion_transformada)
    return corpus_transformado

  def contar_ngramas(self,corpus):
    # Realiza los conteos de ngramas que aparezcan en el corpus
    self.conteos = {}
    if self.n == 1:
      self.conteos[("<s>",)] = len(corpus)
      self.conteos[("<\s>",)] = len(corpus)
    else:
      ngrama_inicio = ["<s>"]*self.n
      self.conteos[tuple(ngrama_inicio)] = len(corpus)

    inicios = []
    for i in range(self.n-1):
      inicios.append("<s>")

    for oracion_normal in corpus:
      oracion = inicios + oracion_normal + ["<\s>"]

      for i in range(len(oracion)-self.n+1):
        ngrama = []
        for j in range(self.n):
          ngrama.append(oracion[i+j])
        ngrama = tuple(ngrama)
        self.conteos[ngrama] = self.conteos.setdefault(ngrama, 0) + 1

    return self.conteos

  def entrenar(self,corpus_orginal):
    # Transforma el corpus dependiendo el vocabulario que se tiene
    corpus = self.transformar_corpus(corpus_orginal)

    # Hace el calculo de probabilidades en base al corpus y el suavizado
    self.numTokens = 0
    for oracion in corpus:
      self.numTokens += len(oracion) + 1

    # Realiza los conteos de nGramas en el corpus transformado
    self.contar_ngramas(corpus)

    # Calcula las probabilidades de los nGramas encontrados, usando el suavizado de Laplace
    self.probabilidades = pd.DataFrame.from_dict(self.conteos,orient="index",columns=["conteos"])
    if self.n == 1:
      self.probabilidades["probabilidad"] = self.probabilidades["conteos"]/self.numTokens
    else:
      self.Nmenos1Gramas = Ngrama(self.n-1,self.vocabulario,self.suavizado)
      self.Nmenos1Gramas.contar_ngramas(corpus)
      if self.suavizado == "Laplace":
        self.probabilidades["probabilidad"] = self.probabilidades.apply(lambda x: laplace(x["conteos"],self.Nmenos1Gramas.conteos[tuple(x.name[:self.n-1])],len(self.vocabulario)),axis = 1)
      elif self.suavizado == "Backoff":
        self.probabilidades["probabilidad"] = self.probabilidades.apply(lambda x: x["conteos"]/self.Nmenos1Gramas.conteos[tuple(x.name[:self.n-1])],axis = 1)
        self.Nmenos1Gramas.entrenar(corpus)
      else:
        print("ERROR: ("+self.suavizado+") No es un tipo de suavizado valido")

    self.probabilidades = self.probabilidades.drop(columns=["conteos"])
    self.probabilidades.drop([tuple(["<s>"]*self.n)],inplace=True) # Se elimina de las probabilidades el nGrama de puros inicios de oración "<s>" (Solo servia para calcular las probabilidades)

  def siguiente_palabra(self,contexto_original):
    # Regresa una palabra al azar dado un contexto y su probabilidad
    # Contexto es una lista de palabras, (limpia pero con posibles palabras que se deban cambiar a <UNK>)
    # Return (Palabra siguiente, prob_palabra_sig, contexto_usado)
    if(len(contexto_original) != self.n-1):
      print("Error: Contexto muy grande")
      return 0,0,None

    contexto = self.transformar_oracion(contexto_original)

    if self.n == 1:
      busqueda = self.probabilidades.sample(1,weights="probabilidad")
      return busqueda.index[0][0], busqueda["probabilidad"][0],None

    # self.n >= 2
    if self.suavizado == "Laplace":
      busqueda = self.probabilidades[self.probabilidades.apply(lambda x: x.name[0:self.n-1] == tuple(contexto), axis=1)]
      if len(busqueda) == 0: #No se encontró el contexto en los conteos de nGramas, todas las palabras siguientes tienen probabilidad 1/|V|
        return self.vocabulario.palabras.sample(1).index[0],1/len(self.vocabulario),tuple(contexto)
      busqueda = busqueda.sample(1,weights="probabilidad")
      return  busqueda.index[0][self.n-1], busqueda["probabilidad"][0],tuple(contexto)
    elif self.suavizado == "Backoff":
      busqueda = self.probabilidades[self.probabilidades.apply(lambda x: x.name[0:self.n-1] == tuple(contexto), axis=1)]
      if len(busqueda) == 0: #No se encontró el contexto en los conteos de nGramas
        # Se usa un contexto menor para obtener la siguiente palabra
        return self.Nmenos1Gramas.siguiente_palabra(contexto[1:])
      busqueda = busqueda.sample(1,weights="probabilidad")
      return  busqueda.index[0][self.n-1], busqueda["probabilidad"][0],tuple(contexto)

    print("ERROR: ("+self.suavizado+") No es un tipo de suavizado valido")
    return 0,0,None

  def generar_oracion(self,longitud_maxima,reemplazar_unk=False):
    # Genera una oración al azar dependiendo las probabilidades de los nGramas calculados
    contexto = ["<s>"]*(self.n-1)

    oracion = []
    extras = []
    for i in range(longitud_maxima):
      palabra,probabilidad,contexto_usado = self.siguiente_palabra(contexto)
      extras.append((probabilidad,contexto_usado))
      if palabra == "<UNK>" and reemplazar_unk:
        palabra = "(UNK)"+self.vocabulario.palabras_unk.sample(1,weights="Frecuencia").index[0]
      oracion.append(palabra)
      if self.n > 1:
        contexto = contexto[1:]+[oracion[-1]]
      if oracion[-1] == "<\s>":
        break

    return oracion,extras

  def probabilidad_ngrama(self,ngrama_original):
    # Regresa la probabilidad de un nGrama
    if(len(ngrama_original) != self.n):
      print("Error: Ngrama muy grande")
      return 0,0,None

    ngrama = self.transformar_oracion(ngrama_original)

    if self.n == 1:
      busqueda = self.probabilidades[self.probabilidades.apply(lambda x: x.name == tuple(ngrama), axis=1)]
      return busqueda["probabilidad"][0],tuple(ngrama)

    # self.n >= 2
    if self.suavizado == "Laplace":
      busqueda = self.probabilidades[self.probabilidades.apply(lambda x: x.name == tuple(ngrama), axis=1)]
      if len(busqueda) == 0: #No se encontró el nGrama C(w_n|contexto) = 0, ahora se busca C(w|contexto)
        conteos_contexto = self.Nmenos1Gramas.conteos.setdefault(tuple(ngrama[:-1]), 0)
        return laplace(0,conteos_contexto,len(self.vocabulario)),tuple(ngrama)
      return  busqueda["probabilidad"][0],tuple(ngrama)
    elif self.suavizado == "Backoff":
      busqueda = self.probabilidades[self.probabilidades.apply(lambda x: x.name == tuple(ngrama), axis=1)]
      if len(busqueda) == 0: #No se encontró el nGrama C(w_n|contexto) = 0, ahora se busca C(w|contexto[:-1])
        # Se usa un contexto menor para obtener la siguiente palabra
        return self.Nmenos1Gramas.probabilidad_ngrama(ngrama[1:])
      return  busqueda["probabilidad"][0],tuple(ngrama)

    print("ERROR: ("+self.suavizado+") No es un tipo de suavizado valido")
    return 0,0,None

  def probabilidad_oracion(self,oracion_original,log = False):
    # Regresa la probabilidad de una oración
    oracion = ["<s>"]*(self.n-1) + oracion_original + ["<\s>"]

    probabilidad = 0 if log else 1
    extras = []
    for i in range(len(oracion)-self.n+1):
        ngrama = []
        for j in range(self.n):
          ngrama.append(oracion[i+j])
        prob_ngrama,ngrama_usado = self.probabilidad_ngrama(ngrama)
        probabilidad = probabilidad+np.log(prob_ngrama) if log else probabilidad*prob_ngrama
        extras.append([prob_ngrama,ngrama_usado])
    return probabilidad,extras,len(oracion)-self.n+1


In [ ]:
def limpiar_oracion(oracion,stopwords,delimeters):
  #Limpia una oracion transformando palabras en minusculas, quitando stopwords y delimeters (puntos, comas, parentesis)
  nueva_oracion = []
  for palabra in oracion:
    palabra_nueva = palabra.lower()
    if not(palabra_nueva in stopwords):
      for delimeter in delimeters:
        palabra_nueva = palabra_nueva.replace(delimeter,"")
      if palabra_nueva != "":
        nueva_oracion.append(palabra_nueva)
  return nueva_oracion

In [ ]:
def imprimir_prob(palabra,prob,contexto):
  print("P("+palabra,end="",sep="")
  if(contexto != None and len(contexto) != 0): # Osea se utilizó algo de contexto
    print("|",end="")
    for j in range(len(contexto)-1):
      print(contexto[j]+",",end="")
    print(contexto[-1],end="")
  print(") = ",prob,sep="")

In [ ]:
stopwords_esp = ["el","no","ellos","si"]
delimeters = [",",".","(",")",";",":","[","]"," ","\t","\n","'",'"']

## Selección del corpus

In [ ]:
#@title ### Obtención del corpus

#@markdown Especifica el nombre del archivo desde el cual se leera el corpus
archivo_corpus = "Canciones zo\xE9 por fila.csv" #@param {type:"string"}
columna_texto = "Letra" #@param {type:"string"}

df_corpus = pd.read_csv(archivo_corpus)
corpus = []
for i in range(len(df_corpus)):
  oracion = df_corpus.iloc[i][columna_texto].split(" ")
  corpus.append(limpiar_oracion(oracion,[],delimeters))


display(Markdown(f'**10 primeras palabras de 3 documentos del corpus:**'))
print(corpus[0][:10])
print(corpus[5][:10])
print(corpus[-1][:10])

## Creación del vocabulario

In [ ]:
#@title ### Obtener el vocabulario del corpus

#@markdown Calcula las frecuencias de cada palabra que ocurra en el corpus incluyendo todos los documentos

# Además se calcula el número de palabras (tokens) totales en el corpus
# Y la frecuencia maxima en el vocabulario
vocabulario_completo = Vocabulario(corpus)
vocabulario_completo.imprimir_datos_vocabulario()

### Reducir el vocabulario

In [ ]:
#@title ### Selección de criterio

#@markdown Dos formas de reducir el vocabulario son mantener en el vocabulario las N palabras con mayor frecuencia o mantener la palabras con una frecuencia mayor o igual a una frecuencia mínima. Las palabras que no cumplan el criterio se cambian por el token "\<UNK\>"
criterio = "N palabras" #@param ["N palabras", "Frecuencia minima"]


lenVocabulario = len(vocabulario_completo.palabras)
frecMaxima = vocabulario_completo.palabras.iloc[0]["Frecuencia"]
nSlider = widgets.IntSlider(value=int(lenVocabulario/2), max=lenVocabulario)
minFrecSlider = widgets.IntSlider(value=int(frecMaxima/2), max=frecMaxima)
if criterio == "N palabras":
  display(Markdown(f'**Selecciona el número N de palabras más frecuentes a conservar:**'))
  display(nSlider)
else:
  display(Markdown(f'**Selecciona la frecuencia mínima de las palabras a conservar:**'))
  display(minFrecSlider)

In [ ]:
#@title Reducción del vocabulario
#@markdown Se reduce el vocabulario dependiendo el criterio y parametro elegido
if criterio == "N palabras":
  vocabularioEsqN = Vocabulario(corpus)
  vocabularioEsqN.reducir_vocabulario(criterio,nSlider.value)
  vocabularioEsqN.imprimir_datos_vocabulario()

else:
  vocabularioEsqMin = Vocabulario(corpus)
  vocabularioEsqMin.reducir_vocabulario(criterio,minFrecSlider.value)
  vocabularioEsqMin.imprimir_datos_vocabulario()


## Entrenamiento de modelos de N-gramas

En un modelo de N-gramas hay varias partes que se pueden realizar de formas diferentes, las formas en que se realizan estas partes constituyen los parametros del modelo.

**Tamaño de los N-gramas**: Define el número de palabras que se tomaran en cuenta para el calculo de probabilidades. Por ejemplo para la oración:

$$veo \;\; una \;\; pelota \;\; roja$$

En un modelo con tamaño de N-gramas = 1, se calculan las probabilidades:

$$P(veo),\;\;P(una),\;\;P(pelota),\;\;P(roja)$$

Para un modelo con tamaño de N-gramas = 3:

$$P(veo|<s>,<s>),\;\;P(una|<s>,veo),\;\;P(pelota|veo,una),\;\;P(roja|una,pelota)$$

**Vocabulario del modelo**: Define las palabras que reconoce el modelo, las palabras que no esten dentro del vocabulario (incluso si no aparecian en el corpus originalmente) seran transformadas al token "\<UNK\>" que representa a las palabras desconocidas (Unknow). Este vocabulario proviene de alguno de los 2 tipos de reducción hechas anteriormente.

![](https://raw.github.com/labsemco/EVIA-UAEM/main/Modelos%20de%20Lenguaje/Notebooks/Imagenes/Palabras%20UNK.png)

**Suavizado**: Define como se trataran los casos en los que la probabilidad de un N-grama sea 0, es decir, cuando un N-grama no aparece dentro del corpus que fue suministrado al modelo para su entrenamiento.

* Laplace: Este suavizado cambia la forma en que se calculan las probabilidades de los N-gramas, agregando un conteo a todos los N-gramas, incluso si no estan en el corpus de entrenamiento, de esta forma evita que las probabilidades se hagan 0 para N-gramas que no esten en el corpus.

 $$p(w_n|w_1^{n-1}) \approx \frac{C(w_n|w_1^{n-1})+1}{\sum_{w\in V} C(w|w_1^{n-1})+|V|}$$

* Backoff: Aproxima la probabilidad de un N-grama no encontrado como la probabilidad de un (N-1)-grama similar, por ejemplo si se busca la probabilidad del N-grama:

$$el\;\;perro\;\;corre\;\;$$
&emsp;&emsp;&ensp;primero intenta buscar dentro del modelo la probabilidad $P(corre|el,perro)$ pero si es cero entonces aproxima la probabilidad como:

$$P(corre|el,perro)\approx P(corre|perro)$$

&emsp;&emsp;&ensp;De forma general se expresa como:

$$P(w_n|w_1^{n-1})\approx P(w_n|w_2^{n-1})\approx \;... \approx P(w_n|w_i^{n-1}) \approx \; ...\approx P(w_n)$$

&emsp;&emsp;&ensp;Mientras $P(w_n|w_i^{n-1})=0$



In [ ]:
#@title Creación de un modelo de N-gramas
#@markdown Selecciona los parametros del modelo de Ngramas a crear

tam_ngrama = 10 #@param {type:"integer"}
reduccion = "Frecuencia minima" #@param ["N palabras", "Frecuencia minima"]
suavizado = "Backoff" #@param ["Laplace", "Backoff"]

if reduccion == "N palabras":
  vocabularioParaModelo = vocabularioEsqN
elif reduccion == "Frecuencia minima":
  vocabularioParaModelo = vocabularioEsqMin
else:
  vocabularioParaModelo = vocabulario_completo

# Diccionario que almacena los nGramas construidos
modelos_entrenados = modelos_entrenados if "modelos_entrenados" in globals() else {}

# Entrena los modelos necesarios de nGramas
modelo_nuevo = Ngrama(tam_ngrama,vocabularioParaModelo,suavizado)
modelo_nuevo.entrenar(corpus)
modelos_entrenados[tuple([tam_ngrama,reduccion,suavizado])] = modelo_nuevo

display(Markdown(f'**10 conteos de {tam_ngrama}-gramas al azar**'))
for i in range(10):
  res = key, val = random.choice(list(modelo_nuevo.conteos.items()))
  print(res[0], "aparece",val, "veces")

display(Markdown(f'**10 probabilidades de {tam_ngrama}-gramas al azar**'))
print(modelo_nuevo.probabilidades.sample(10))

In [ ]:
#@title Busqueda de N-gramas dentro del modelo creado
N_grama = "'hay', 'nada', 'que', 'pueda', 'perder', 'que', 'no', 'pueda', 'ser', 'que'" #@param {type:"string"}

if N_grama != "":
  N_grama_lista = N_grama.split(" ")
else:
  N_grama_lista = []

if "modelo_nuevo" in globals():
  if len(N_grama_lista) != modelo_nuevo.n:
    display(Markdown(f'**ERROR: El N-grama debe ser de tamaño {modelo_nuevo.n} no de {len(N_grama_lista)}**'))
  else:
    display(Markdown(f'**N-grama transformado:**'))
    print(modelo_nuevo.transformar_oracion(N_grama_lista))

    prob,extras = modelo_nuevo.probabilidad_ngrama(N_grama_lista)

    display(Markdown(f'**Probabilidad asignada al N-grama:**'))
    print(prob)

    display(Markdown(f'**N-grama calculado:**'))
    imprimir_prob(extras[-1],prob,extras[:-1])
else:
  display(Markdown(f'**ERROR: No hay ningún modelo de N-gramas creado**'))

## Uso de los modelos de N-gramas

In [ ]:
#@title Selección del modelo a usar

lista_modelos = [ key for key in modelos_entrenados]
lista_modelos_str = [ str(key) for key in modelos_entrenados]

seleccion_modelo = widgets.Dropdown(
    options=lista_modelos_str,
    value=lista_modelos_str[0],
    disabled=False,
)

display(Markdown(f'**Selecciona el modelo a usar:**'))
display(seleccion_modelo)

In [ ]:
#@title Generación de oraciones aleatorias
#@markdown Genera una oración al azar de longitud menor o igual a M usando las probabilidades calculadas en el modelo de N-gramas

long_maxima = 10 #@param {type:"integer"}
reemplazar_UNK = False #@param {type:"boolean"}
mostrar_probabilidades = False #@param {type:"boolean"}

modelo = modelos_entrenados[lista_modelos[seleccion_modelo.index]]

oracion,extras = modelo.generar_oracion(long_maxima,reemplazar_UNK)

display(Markdown(f'**Oración generada al azar usando el modelo de N-gramas entrenado:**'))
for elemento in oracion:
  print(elemento,end=" ")
print()

if(mostrar_probabilidades):
  for i in range(len(extras)):
    imprimir_prob(oracion[i],extras[i][0],extras[i][1])

In [ ]:
#@title Obtención de palabras siguientes dado un contexto
#@markdown Da una palabra al azar que puede aparecer dado un contexto
contexto = "casi nunca dice que yo estoy enamorado de " #@param {type:"string"}

if contexto != "":
  oracion_contexto = ["<s>"]*(modelo.n-1) + contexto.split(" ")
else:
  oracion_contexto = ["<s>"]*(modelo.n-1)

modelo = modelos_entrenados[lista_modelos[seleccion_modelo.index]]

display(Markdown(f'**Oración transformada:**'))
print(modelo.transformar_oracion(oracion_contexto[modelo.n-1:]))

sig_palabra,prob,contexto_usado = modelo.siguiente_palabra(oracion_contexto[len(oracion_contexto)-modelo.n+1:])

display(Markdown(f'**Siguiente palabra escogida al azar con su respectiva probabilidad:**'))
print(sig_palabra)
imprimir_prob(sig_palabra,prob,contexto_usado)

In [ ]:
#@title Probabilidad asignada a una oración
#@markdown Da la probabilidad que el modelo le asigna a la oración dada
oracion = "casi nunca nadie dice que yo estoy enamorado" #@param {type:"string"}
mostrar_ngramas = False #@param {type:"boolean"}

if oracion != "":
  oracion_lista = oracion.split(" ")
else:
  oracion_lista = []

modelo = modelos_entrenados[lista_modelos[seleccion_modelo.index]]

display(Markdown(f'**Oración transformada:**'))
print(modelo.transformar_oracion(oracion_lista))

prob,extras,_ = modelo.probabilidad_oracion(oracion_lista)

display(Markdown(f'**Probabilidad asignada a la oración:**'))
print(prob)

if(mostrar_ngramas):
  display(Markdown(f'**N-gramas calculados:**'))
  for i in range(len(extras)):
    imprimir_prob(extras[i][1][-1],extras[i][0],extras[i][1][:-1])

## Comparación de los modelos de N-gramas

In [ ]:
#@title Probabilidad asignada a una oración
#@markdown Compara la probabilidad que le asignan los modelos entrenados sobre un mismo vocabulario a la oración dada
oracion = "casi nunca nadie dice que yo estoy enamorado" #@param {type:"string"}
reduccion = "Frecuencia minima" #@param ["N palabras", "Frecuencia minima"]
log_probabilidad = False #@param {type:"boolean"}
mostrar_ngramas = False #@param {type:"boolean"}

espacio = "\t\t\t" if reduccion == "Frecuencia minima" else "\t\t"

if oracion != "":
  oracion_lista = oracion.split(" ")
else:
  oracion_lista = []

modelos = [(key,modelos_entrenados[key]) for key in modelos_entrenados if key[1] == reduccion]
modelos.sort(key=lambda x: x[0][2])
if len(modelos) == 0:
  display(Markdown(f'**ERROR:** No hay modelos creados bajo ese vocabulario'))
else:
  probabilidades = []
  list_extras = []
  for key,modelo in modelos:
    prob,extras,_ = modelo.probabilidad_oracion(oracion_lista,log_probabilidad)
    probabilidades.append(prob)
    list_extras.append(extras)

  display(Markdown(f'**Oración transformada:**'))
  print(modelo.transformar_oracion(oracion_lista))

  display(Markdown(f'**Probabilidades por cada modelo**'))
  print("Modelo:        ",end="")
  for key,modelo in modelos:
    print(key,end="\t")
  if log_probabilidad:
    print("\nLog(Probabilidad):      ",end="")
  else:
    print("\nProbabilidad:      ",end="")
  for i in range(len(probabilidades)):
    print(probabilidades[i],end=espacio)

  text_prob = "Log(Probabilidad)" if log_probabilidad else "Probabilidad"

  display(Markdown(f'**Mapa de calor de las probabilidades por cada modelo**'))
  df = pd.DataFrame({"Modelo": [(key[0],key[2]) for key,modelo in modelos],
                   text_prob: probabilidades})
  df.set_index("Modelo", inplace=True)
  ax = sns.heatmap(df, annot=True, fmt="g", cmap='Reds',vmin=min(probabilidades), vmax=max(probabilidades))
  plt.show()

  if mostrar_ngramas:
    for i in range(len(modelos)):
      display(Markdown(f'**N-gramas calculados del modelo {modelos[i][0]}**'))
      extras = list_extras[i]
      for i in range(len(extras)):
        imprimir_prob(extras[i][1][-1],extras[i][0],extras[i][1][:-1])


## Bibliografía

* Jurafsky, D. (2018). *Speech and Language Processing: An Introduction to Natural Language Processing,
Computational Linguistics, and Speech Recognition*.
* Manning, C. (2000). *Foundations of Statistical Natural Language Processing*. London, England: The MIT Press.